## Compiling into Booleans and Calling SAT-solver

The following code is an adaptation of the OR-Tools tutorial for SAT-solving the N-Queens problem.

The original tutorial does not compile the variables into booleans and our approach does as intended.

In [1]:
import sys
import time
from ortools.sat.python import cp_model

In [3]:
board_size=int(input())

This class is to override the OR-Tools API to print the desired solution.

In [7]:
class NQueenSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, queens: list[list[cp_model.BoolVarT]]):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__queens = queens
        self.__solution_count = 0
        self.__start_time = time.time()

    @property
    def solution_count(self) -> int:
        return self.__solution_count

    def on_solution_callback(self):
        current_time = time.time()
        print(
            f"Solution {self.__solution_count}, "
            f"time = {current_time - self.__start_time} s"
        )
        self.__solution_count += 1

        all_queens = range(len(self.__queens))
        for i in all_queens:
            for j in all_queens:
                if self.value(self.__queens[j][i]):
                    # There is a queen in column j, row i.
                    print("Q", end=" ")
                else:
                    print("_", end=" ")
            print()
        print()

In [14]:
model = cp_model.CpModel()

Compiling the variables into booleans: each cell of the board is a boolean for the presence of a queen.

In [15]:
# There are `board_size` number of variables, one for a queen in each column
# of the board. The value of each variable is the row that the queen is in.
queens=[]
for i in range(board_size):
    queens.append([])
    for j in range(board_size):
        queens[i].append(model.new_bool_var(f"x_{i}_{j}"))

display(queens)

[[x_0_0(0..1), x_0_1(0..1), x_0_2(0..1), x_0_3(0..1)],
 [x_1_0(0..1), x_1_1(0..1), x_1_2(0..1), x_1_3(0..1)],
 [x_2_0(0..1), x_2_1(0..1), x_2_2(0..1), x_2_3(0..1)],
 [x_3_0(0..1), x_3_1(0..1), x_3_2(0..1), x_3_3(0..1)]]

The constraint formulation is harder for a boolean compiled SAT Problem than to a regular constraint problem, due to row/column/diagonal index proprerties not being obvious to leverage.

There are explicit constraints for every diagonal in a square matrix in the following code.

In [16]:
# All rows must be different.
for i in range(board_size):
    model.add_at_most_one(queens[i])
    model.add_at_least_one(queens[i])

# No two queens can be on the same diagonal.
for i in range(board_size-1):
    model.add_at_most_one(queens[i+k][k] for k in range(0,board_size-i))
for i in range(1,board_size):
    model.add_at_most_one(queens[i-k][k] for k in range(0,i))
for j in range(1,board_size):
    model.add_at_most_one(queens[k][j+k] for k in range(0,board_size-j))
for j in range(board_size-1):
    model.add_at_most_one(queens[k][j-k] for k in range(0,j))

Finally we call the SAT-solver, print the solutions and some statistics.

In [19]:
# Solve the model.
solver = cp_model.CpSolver()
solution_printer = NQueenSolutionPrinter(queens)
solver.parameters.enumerate_all_solutions = True
solver.solve(model, solution_printer)

Solution 0, time = 0.0010924339294433594 s
_ Q Q Q 
Q _ _ _ 
_ _ _ _ 
_ _ _ _ 

Solution 1, time = 0.0012545585632324219 s
_ Q Q Q 
_ _ _ _ 
Q _ _ _ 
_ _ _ _ 

Solution 2, time = 0.0015845298767089844 s
_ Q Q Q 
_ _ _ _ 
_ _ _ _ 
Q _ _ _ 

Solution 3, time = 0.0016870498657226562 s
_ _ Q Q 
_ _ _ _ 
_ _ _ _ 
Q Q _ _ 

Solution 4, time = 0.001783609390258789 s
_ _ Q Q 
Q _ _ _ 
_ _ _ _ 
_ Q _ _ 

Solution 5, time = 0.0018620491027832031 s
_ _ _ Q 
Q _ _ _ 
_ _ Q _ 
_ Q _ _ 

Solution 6, time = 0.001941680908203125 s
_ _ _ Q 
_ _ _ _ 
_ _ Q _ 
Q Q _ _ 

Solution 7, time = 0.0020143985748291016 s
_ Q _ Q 
_ _ _ _ 
_ _ Q _ 
Q _ _ _ 

Solution 8, time = 0.002087116241455078 s
_ Q _ Q 
Q _ _ _ 
_ _ Q _ 
_ _ _ _ 

Solution 9, time = 0.0021581649780273438 s
_ Q _ Q 
_ _ _ _ 
Q _ Q _ 
_ _ _ _ 

Solution 10, time = 0.0022323131561279297 s
_ Q _ Q 
_ _ _ _ 
Q _ _ _ 
_ _ Q _ 

Solution 11, time = 0.0023229122161865234 s
_ Q _ Q 
_ _ _ _ 
_ _ _ _ 
Q _ Q _ 

Solution 12, time = 0.002413034439086914 

4

In [20]:
# Statistics.
print("\nStatistics")
print(f"  conflicts      : {solver.num_conflicts}")
print(f"  branches       : {solver.num_branches}")
print(f"  wall time      : {solver.wall_time} s")
print(f"  solutions found: {solution_printer.solution_count}")


Statistics
  conflicts      : 0
  branches       : 388
  wall time      : 0.006017914 s
  solutions found: 61


## Constraint Problem Solver

This implementation of a Constraint Problem Solver follows the OR-Tools tutorial for CP Solving the N-Queens problem.

In [38]:
import sys
from ortools.constraint_solver import pywrapcp

In [39]:
solver = pywrapcp.Solver("n-queens")

In [ ]:
board_size=int(input())

In [40]:
# The array index is the column, and the value is the row.
queens = [solver.IntVar(0, board_size - 1, f"x{i}") for i in range(board_size)]

In [41]:
# All rows must be different.
solver.Add(solver.AllDifferent(queens))

# No two queens can be on the same diagonal.
solver.Add(solver.AllDifferent([queens[i] + i for i in range(board_size)]))
solver.Add(solver.AllDifferent([queens[i] - i for i in range(board_size)]))

In [42]:
db = solver.Phase(queens, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)

In [43]:
# Iterates through the solutions, displaying each.
num_solutions = 0
solver.NewSearch(db)
while solver.NextSolution():
        # Displays the solution just computed.
    for i in range(board_size):
        for j in range(board_size):
            if queens[j].Value() == i:
                    # There is a queen in column j, row i.
                print("Q", end=" ")
            else:
                print("_", end=" ")
        print()
    print()
    num_solutions += 1
solver.EndSearch()

_ _ Q _ 
Q _ _ _ 
_ _ _ Q 
_ Q _ _ 

_ Q _ _ 
_ _ _ Q 
Q _ _ _ 
_ _ Q _ 



In [44]:
 # Statistics.
print("\nStatistics")
print(f"  failures: {solver.Failures()}")
print(f"  branches: {solver.Branches()}")
print(f"  wall time: {solver.WallTime()} ms")
print(f"  Solutions found: {num_solutions}")



Statistics
  failures: 4
  branches: 10
  wall time: 113721 ms
  Solutions found: 2
